In [ ]:
from src.data import Dataset, available_datasets
from src import workflow

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

In [ ]:
dataset_list = workflow.available_datasets()
dataset_list

# Overall task: 

Train a supervised model on the lvq-pak. Try three different techniques, and pick the one with the best accuracy score.

# Step 0: Import data

In [ ]:
ds_test = Dataset.load('lvq-pak_test')

In [ ]:
ds_test.data.shape

In [ ]:
ds_train = Dataset.load('lvq-pak_train')

In [ ]:
ds_train.data.shape

In [ ]:
ds_train.target

In [ ]:
print(ds_train.DESCR)

## Prototype the workflow in a notebook!

Let's start with one algorithm!

`make train`

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
model = LinearSVC(random_state=42)

In [ ]:
model.fit(ds_train.data, ds_train.target)

In [ ]:
%%time
model = LinearSVC(random_state=42, max_iter=200000)
model.fit(ds_train.data, ds_train.target)

## Use it to predict
`make predict`

In [ ]:
p_test = model.predict(ds_test.data);
p_test

## Test the quality of the prediction
`make analysis`

In [ ]:
model.score(ds_test.data, ds_test.target)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(ds_test.target, p_test)

## Next: 
* automate the basic workflow
* compare 3 different algorithms for our Swedish Chef

# Step 1: `make train`

## Add our algorithm to available_algorithms

There are currently no available algorithms.

In [ ]:
workflow.available_algorithms()

To add an algorithm, add  a key:value pair to the dict `_ALGORITHMS` in `src/models/algorithms.py`.

For example, add
```
'linearSVC': LinearSVC()
```
to the `_ALGORITHMS` dict, and add
```
from sklearn.svm import LinearSVC
```
to the top of the file.

Also, add `linearSVC` to the docstring of `available_algorithms`.

In [ ]:
workflow.available_algorithms()

In [ ]:
help(workflow.available_algorithms)

Now we're in a position where the `make train` script can run using `linearSVC`. 

In [ ]:
!cd .. && make -n train

You'll notice that `make train` takes a `models/model_list.json` as input.
```
## train / fit / build models
train: models/model_list.json
	$(PYTHON_INTERPRETER) -m src.models.train_model model_list.json
```

Under the hood, a `model_list.json` is a list of dicts, where each dict specifices a combination of:
* `dataset_name`: A valid dataset name from `available_datasets`
* `algorithm_name`: A valid dataset name from `available_algorithms`
* `algorithm_params`: A dictionary of parameters to use when running the specified `algorithm`
* `run_number`: (optional, default 1) A unique integer used to distinguish between different builds with otherwise identical parameters

We don't need to know this, as we will use helper functions in `workflow` to build it.

In [ ]:
workflow.add_model(dataset_name='lvq-pak_train',
                   algorithm_name="linearSVC",
                   algorithm_params={'random_state': 42, 'max_iter': 200000})

In [ ]:
workflow.get_model_list()

Now running `make train` will train `LinearSVC` on `lvq-pak` with the specified parameters.

Alternately, we can run `workflow.build_models()`.

The output will be:
* A trained model in `models/trained_models`
* A json file `models/trained_models.json` that keeps track of the models that we've trained

In [ ]:
workflow.build_models()

In [ ]:
!cd .. && make train

## TODO: Caching! Then, checking against existing files and metadata and looking for caching! (note: will need a force parameter eventually)

## TODO: Don't overwrite the trained_models.json, append to it (as long as the files are still there) --- add call to available_models in build_models and give it a force option.

### Let's take a look at the output from `make train`

In [ ]:
from src.paths import trained_model_path
from src.utils import list_dir
from src.utils import load_json

In [ ]:
workflow.available_models()

In [ ]:
# load up the trained model
from src.models.train import load_model

tm, tm_metadata = load_model(model_name='linearSVC_lvq-pak_train_1', model_path=trained_model_path)

In [ ]:
tm

In [ ]:
tm_metadata

In [ ]:
ds = Dataset.load('lvq-pak_train')
ds.DATA_HASH

## TODO: explore the effects of caching once it's implemented

Any algorithm will work that: 
* is a subclass of the sklearn `BaseEstimator` class (needed for setting and getting params)
* has a `fit` method (needed for `make train`)
* has either a `predict` method (supervised) or a `transform` method (unsupervised) (needed for `make predict`)

We will see how things work in the unsupervised case in the next example. 

Note that an **algorithm** here can be a combination of "algorithms" as long as that combination is a `BaseEstimator` with the above methods. For example, you can use an sklearn pipeline, or an sklearn meta estimator like GridSearchCV as an algorithm. 

If your algorithm of choice is **not yet** a `BaseEstimator` with the appropriate API, it is fairly easy to wrap it to be used in this way. While we won't have time to cover an example of this during the in-person part of this tutorial, the EDA Text Embedding (advanced usage tutorial) has an example of how to do this with gensim's FastText model.



# Step 2: `make predict`

```
## predict / transform / run experiments
predict: models/predict_list.json
	$(PYTHON_INTERPRETER) -m src.models.predict_model predict_list.json
```

Similar to `models_list.json` in `predict_list.json` we specify the dataset to operate on, and in this case, the `trained_model` to apply to the given dataset. Again, we do this using the `workflow` module.


A `predict_list.json` is a list of dicts, where each dict specifices a combination of:
* `dataset_name`: A valid dataset name from `available_datasets`
* `dataset_params`: A dictionary of parameters that can be passed to `load_dataset()` with the specified `dataset`
* `model_name`: A valid dataset name from `available_trained_models` (aka. a key name in `trained_models.json`
* `is_supervised`: Whether to use the `predict` (supervised) or `transform` (unsupervised) method


Let's use the test set here to do the prediction.

In [ ]:
workflow.add_prediction(dataset_name='lvq-pak_test', model_name='linearSVC_lvq-pak_train_1', is_supervised=True)

In [ ]:
workflow.get_prediction_list()

In [ ]:
workflow.run_predictions(predict_file='predict_list.json')

In [ ]:
!cd .. && LOGLEVEL=INFO make predict

In [ ]:
workflow.available_predictions()

In [ ]:
from src.paths import model_output_path

#### Note: Predictions are just Datasets tagged with experiment data

In [ ]:
predict_ds = Dataset.load('linearSVC_lvq-pak_train_1_exp_lvq-pak_test_1', data_path=model_output_path)

In [ ]:
predict_ds.data.shape

In [ ]:
predict_ds.metadata['experiment']

In [ ]:
## Check that our prediction matches
all(predict_ds.data == p_test)

# Step 3: `make summary`

## TODO: Add all of this to the standard workflow

In [ ]:
summarizer_list = [{
    'summarizer_name': 'supervised_score_df',
    'summarizer_params': {}
}
]

In [ ]:
from src.paths import reports_path
from src.utils import save_json

In [ ]:
save_json(reports_path / 'summary_list.json', summarizer_list)

In [ ]:
!cd .. && make summary

## TODO: Outputs available via available_sumamries

## TODO: Add caching of the summary dfs to know if you're about to overwrite one

# An Aside: Add other algorithms


### Exercise: Add GradientBoostingClassifier and some other sklearn Classifier of your choice

### Advanced Exercise: Use GridSearchCV applied to your classifier of choice as the 3rd alg

In [ ]:
workflow.add_model(
    dataset_name = 'lvq-pak_train',
    algorithm_name = 'GradientBoostingClassifier',
    algorithm_params = {'random_state': 42}    
)

In [ ]:
### Add your choice of classifier here

In [ ]:
### Take a look to see what's there
workflow.get_model_list()

In [ ]:
workflow.get_model_list()

In [ ]:
workflow.available_algorithms(keys_only=False)

In [ ]:
!cd .. && make train

In [ ]:
workflow.available_models()

In [ ]:
workflow.get_prediction_list()

In [ ]:
## Set up predictions using all of the available models
for tm in workflow.available_models():
    workflow.add_prediction(
        dataset_name = 'lvq-pak_test',
        model_name = tm,
        is_supervised = True,
    )

In [ ]:
workflow.get_prediction_list()

In [ ]:
!cd .. && LOGLEVEL=DEBUG make predict

In [ ]:
workflow.available_predictions()

The default for running the the summary df is to run on all available predictions. We have nothing more that we have to add to our existing script to get all the new scores. 

In [ ]:
!cd .. && make summary

## TODO: add the next part to a `load_summary` call

In [ ]:
import pandas as pd

In [ ]:
from src.paths import summary_path

In [ ]:
list_dir(summary_path)

In [ ]:
pd.DataFrame.from_csv(summary_path / 'supervised_score_df')

# Step 5: `make report`

### Bonus Exercise: Turn your dataframe into a table that you can share with the Swedish Chef in a report that justifies why you picked the model that you gave to him

This is mostly a creative writing exercise, so we'll skip it here...however, if you're way ahead of the crowd, take a stab at creating the necessary Makefile target and corresponding scripts to create your report!

# Step 6: `make publish`

For this task, publishing involves:
* versioning and releasing the code that we used to create the delivered model
* versioning and sharing the supporting report

We'll cover how to do both of these briefly at the end of the tutorial time permitting.

## TODO: Figure out where and how to include a "lesson" on random_state
